In [1]:
import os
import random
import skimage.data
from skimage import io
import skimage.transform as skt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

In [2]:
# submap_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression/submap"

# # for i in range(4):
# i = 1
# submap1 = cv2.imread(os.path.join(submap_dir,str(i) + '.pgm'))
    
    
# submap1 = cv2.resize(submap1,(448,448))
# # submap2 = skimage.data.imread(os.path.join(submap_dir,'1.pgm'))
# # img = cv2.merge([submap1,submap2])
# cv2.imshow("img",submap1)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# print(np.shape(submap1))

In [3]:
label_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression"
pic_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression/picture"
submap_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression/submap"

train_txt = os.path.join(label_dir, "label.txt")
count = len(open(train_txt, 'r').readlines())
print(count)
file = open(train_txt, 'r')
s = file.readlines()
all_images = []
all_labels = []

for i in range(0, count-2):
#     image1 = skimage.data.imread(os.path.join(pic_dir, s[i].split(";")[0]))
    submap1 = cv2.imread(os.path.join(submap_dir,str(i) + '.pgm'))
    submap1 = cv2.resize(submap1,(448,448))
    submap2 = cv2.imread(os.path.join(submap_dir,str(i+1) + '.pgm'))
    submap2 = cv2.resize(submap2,(448,448))
    label = s[i].split(";")[1:3]
    label = list(map(float,label))
#     image2 = skimage.data.imread(os.path.join(pic_dir, s[i].split(";")[0]))
#     image3 = skimage.data.imread(os.path.join(pic_dir, s[i+1].split(";")[0]))
#     image4 = skimage.data.imread(os.path.join(pic_dir, s[i+2].split(";")[0]))
    img = cv2.merge([submap1,submap2])
#     img = cv2.merge([image1,image2,image3,image4])

    all_images.append(img)
    all_labels.append(label)
    
print(np.shape(all_images))
print(np.shape(all_labels))

4026
(4024, 448, 448, 6)
(4024, 2)


In [4]:
def get_train_data_batch(num):
    idx = np.arange(0, count-2)
    np.random.shuffle(idx)
    data_idx = idx[:num]

    labels = []
    images = []

    for i in range(num):
        images.append(all_images[data_idx[i]])
        labels.append(all_labels[data_idx[i]])

    return np.asarray(images), np.asarray(labels)
X,Y = get_train_data_batch(10)
print(Y)
print(np.shape(X))
print(np.shape(Y))

[[-2.30153998e+00 -1.04930384e+01]
 [ 1.97840992e+01  9.58769297e-01]
 [-3.05771810e+01  1.06716167e+01]
 [-2.87421323e+01 -2.19219096e+01]
 [ 1.49299090e+00  2.74776712e-02]
 [-3.08408373e+01  9.18608482e+00]
 [-2.87829100e+01 -1.63245994e+01]
 [-4.65354138e+00 -9.68551880e+00]
 [ 9.30219858e+00 -9.85694935e+00]
 [-1.19117154e+01 -6.85686176e+00]]
(10, 448, 448, 6)
(10, 2)


In [5]:
with tf.name_scope("input"):
    input_image = tf.placeholder("float", [None, 448,448,6],name='input_image')
#     input_image = tf.pad(image,np.array([[0, 0], [3, 3], [3, 3], [0, 0]]))
    y_ = tf.placeholder("float", [None, 2],name='y_') 

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")


W_conv0 = weight_variable([7, 7, 6, 64])
b_conv0 = bias_variable([64])

h_conv0 = tf.nn.leaky_relu(tf.nn.conv2d(input_image, W_conv0, strides=[1,1,1,1], padding="SAME") + b_conv0)
h_pool0 = max_pool_2x2(h_conv0)   #输出变成224*224*64 [None,224,224,64]

# #第一层卷积层
# #参数的前两维为卷积核的维度，第三个参数为当前层的深度，第四个为输出到层的深度

W_conv1 = weight_variable([7, 7, 64, 64])
b_conv1 = bias_variable([64])

h_conv1 = tf.nn.leaky_relu(conv2d(h_pool0, W_conv1) + b_conv1)
# h_conv1 = tf.nn.relu(tf.nn.conv2d(input_image, W_conv1, strides=[1,2,2,1], padding="SAME") + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)   #输出变成112*112*64   [None,112,112,64]

# #第一层卷积层
# #参数的前两维为卷积核的维度，第三个参数为当前层的深度，第四个为输出到层的深度
# W_conv1 = weight_variable([7, 7, 6, 64])
# b_conv1 = bias_variable([64])

# h_conv1 = tf.nn.relu(tf.nn.conv2d(input_image, W_conv1,strides=[1,2,2,1],padding="SAME") + b_conv1)
# h_pool1 = max_pool_2x2(h_conv1)   #输出变成112*112*64

#第二层卷积层
W_conv2 = weight_variable([3, 3, 64, 192])
b_conv2 = bias_variable([192])

h_conv2 = tf.nn.leaky_relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)  #输出变为56*56*192


#第三层卷积层
W_conv3 = weight_variable([3, 3, 192, 128])
b_conv3 = bias_variable([128])

h_conv3 = tf.nn.leaky_relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)   #输出为28*28*128


#第四层卷积层
W_conv4 = weight_variable([3, 3, 128, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.leaky_relu(conv2d(h_pool3, W_conv4) + b_conv4)
h_pool4 = max_pool_2x2(h_conv4)   #输出变为14*14*64


#密集连接层1
W_fc1 = weight_variable([14*14*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool4, [-1, 14*14*64])
h_fc1 = tf.nn.leaky_relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)   #输出为96*1024

keep_prob = tf.placeholder("float")
#relu函数作用是求出max(h_fc1,0)
h_fc1_drop1 = tf.nn.dropout(h_fc1, keep_prob)   #输出为96*1024


#密集连接层2
W_fc2 = weight_variable([1024, 512])
b_fc2 = bias_variable([512])

h_fc2 = tf.nn.leaky_relu(tf.matmul(h_fc1_drop1, W_fc2) + b_fc2)

#relu函数作用是求出max(h_fc1,0)
h_fc1_drop2 = tf.nn.dropout(h_fc2, keep_prob)

#输出层softmax层
W_fc3 = weight_variable([512, 2])

y_conv = tf.matmul(h_fc1_drop2, W_fc3)
# tf.summary.histogram('y_conv',y_conv)

# batch_x = images[1:3]
# batch_y = labels[1:3]
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     print("shape of outputs is:",sess.run(tf.shape(y_conv),feed_dict={input_image:batch_x, 
#                                                                            y_:batch_y,keep_prob:1.0}))
#         print("learning rate is :",sess.run(learning_rate,feed_dict = {global_step:i}))


starter_learning_rate = 0.0002
steps_per_decay = 1000
decay_factor = 0.98
 
global_step = tf.Variable(tf.constant(0))
learning_rate = tf.train.exponential_decay(learning_rate = starter_learning_rate,
                                           global_step = global_step,
                                           decay_steps = steps_per_decay,
                                           decay_rate = decay_factor,
                                           staircase = True
                                           )

  
delta=tf.constant(1.25)
#huber损失函数
loss = tf.reduce_mean(tf.multiply(tf.square(delta),
                                          tf.sqrt(1. + tf.square((y_ - y_conv)/delta)) - 1.))

# cross_entropy = tf.reduce_sum(tf.square(y_-y_conv)) 
tf.summary.scalar("loss",loss)

train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
# train_step = tf.train.AdadeltaOptimizer(learning_rate).minimize(cross_entropy)
# train_step = tf.train.RMSPropOptimizer(learning_rate).minimize(cross_entropy)

# correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# tf.summary.scalar("accuracy", accuracy)
merged = tf.summary.merge_all()

batch_size = 8               
checkpointsPath = "./checkpoints/"
reload = False

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #保存训练好的模型一遍下次使用
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter("logs_dir/", sess.graph)
    
    if not os.path.exists(checkpointsPath):
        os.mkdir(checkpointsPath)
    
    if reload:
        checkPoint = tf.train.get_checkpoint_state(checkpointsPath)
        if checkPoint and checkPoint.model_checkpoint_path:
            saver.restore(sess,checkPoint.model_checkpoint_path)
            print("restored %s" % checkPoint.model_checkpoint_path)
        else:
            print("no checkpoint found!")
        
    for i in tqdm(range(60000)):
        batch_x,batch_y = get_train_data_batch(batch_size)
        batch_y = batch_y*1000
        summary,_ = sess.run([merged,train_step],feed_dict={global_step:i,
                                                            input_image:batch_x, 
                                                            y_:batch_y, 
                                                            keep_prob:0.8})
        writer.add_summary(summary, i)
        if (i % 1000) == 0 & (i != 0):
            print("save model")
            print("learning_rate is:\n",sess.run(learning_rate,feed_dict = {global_step:i}))
            print("real location is:\n",batch_y)
            print("prediction is:\n",sess.run(y_conv,feed_dict={input_image:batch_x, y_:batch_y,keep_prob:1.0}))
#             saver.save(sess, os.path.join(checkpointsPath,"saved_net.ckpt"), global_step=i)
            print("loss is:",sess.run(loss,feed_dict={input_image:batch_x, y_:batch_y, keep_prob:1.0}))
    print("done!")


  0%|          | 0/60000 [00:00<?, ?it/s]

save model
learning_rate is:
 0.0002
real location is:
 [[  2807.11369868 -10623.8392915 ]
 [ 30893.2155665   -4675.28437805]
 [ -3662.33991761   -634.95718102]
 [-28541.1234698   -5966.62581026]
 [ 12567.7303482   -7174.55263297]
 [ 30507.097668    -1753.86643243]
 [-19927.4997337   -7001.21353822]
 [ 18743.0708866   -9576.38234625]]
prediction is:
 [[ 529079.75 -313813.56]
 [ 530801.   -312222.84]
 [ 525034.75 -327428.94]
 [ 491353.53 -309008.78]
 [ 522349.62 -309323.66]
 [ 524897.6  -311715.62]
 [ 520661.66 -313349.88]
 [ 537118.9  -322049.38]]


  0%|          | 1/60000 [00:02<42:09:34,  2.53s/it]

loss is: 516359.0


  2%|▏         | 1000/60000 [05:45<5:30:52,  2.97it/s]

...

save model
learning_rate is:
 6.07252e-05
real location is:
 [[ 24397.6122839    1032.7605676 ]
 [  1990.71429929    287.98204429]
 [-30353.1170969    2563.38381039]
 [ -4448.44085744 -10160.0563356 ]
 [-26135.3149102   -7547.18497683]
 [ 10558.1903528     798.99322222]
 [-28561.3525985   -7786.24121767]
 [ -7483.83930787  -4801.90445303]]
prediction is:
 [[ 24006.797      436.88922]
 [  1989.757     -318.97333]
 [-30526.664     2323.354  ]
 [ -4627.8916   -9943.274  ]
 [-25252.203    -7105.815  ]
 [ 10085.86       619.79517]
 [-28320.686    -7637.328  ]
 [ -7448.171    -4873.0176 ]]


 98%|█████████▊| 59001/60000 [5:30:32<06:44,  2.47it/s]

loss is: 379.48972


100%|██████████| 60000/60000 [5:36:08<00:00,  2.95it/s]

done!
